# Week02 - Advanced Integration Techniques

본 노트북은 고급 프롬프트 엔지니어링 통합 기법들을 실습합니다.

## 다룰 기법들
1. **Function/Tool Calling**: 외부 함수/도구 호출 및 결과 활용
2. **Multiple Chains**: 파이프라인 연결 및 단계별 처리
3. **Meta-Prompting**: 프롬프트 자체를 진단하고 개선
4. **APE (Automatic Prompt Engineering)**: 프롬프트 자동 생성 및 평가

In [1]:
# 필요한 라이브러리 설치 및 import
import subprocess
import json
import re
import time
import random
import os
import sys
from dotenv import load_dotenv
from typing import Dict, List, Optional, Any, Union
from io import StringIO

# .env 파일 로드
load_dotenv()

# OpenAI 라이브러리
try:
    from openai import OpenAI, APIError, RateLimitError
except ImportError:
    !pip install openai
    from openai import OpenAI, APIError, RateLimitError

# 설정
client = OpenAI()  # 환경변수에서 API 키 자동 로드

# 헬퍼 함수들
def run_ollama(model: str, prompt: str) -> str:
    """Ollama 모델 실행"""
    try:
        result = subprocess.run(
            ["ollama", "run", model],
            input=prompt,
            text=True,
            capture_output=True,
            timeout=60
        )
        return result.stdout.strip()
    except subprocess.TimeoutExpired:
        return "Error: Timeout"
    except Exception as e:
        return f"Error: {str(e)}"

def run_openai(prompt: str, model: str = "gpt-4o-mini", **kwargs) -> str:
    """OpenAI 모델 실행"""
    try:
        response = client.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": prompt}],
            **kwargs
        )
        return response.choices[0].message.content
    except Exception as e:
        return f"Error: {str(e)}"

def safe_json_parse(text: str) -> Union[dict, None]:
    """안전한 JSON 파싱"""
    try:
        # 코드 블록 제거
        clean_text = re.sub(r'```json\s*|```\s*', '', text).strip()
        return json.loads(clean_text)
    except:
        return None

print("라이브러리 로드 완료!")

라이브러리 로드 완료!


## 1. Function/Tool Calling

**핵심 아이디어**: LLM이 외부 함수/도구를 "이름+파라미터"로 호출 → 실제 결과를 받아 최종 답변에 반영

**설계 팁**:
- 스키마 명세: `name`, `description`, `parameters(JSON Schema)` 명확히
- Idempotency: 재시도 시 중복 실행 방지
- 타임아웃/백오프: 툴 실패·지연 대응
- 보안: 파라미터 화이트리스트, 출력 검증

In [2]:
# 1-1. Ollama 툴 시뮬레이션
def simulate_tool_calling_ollama():
    """Ollama로 툴 호출 시뮬레이션"""
    
    # 1) 모델에게 툴 호출 JSON 생성하도록 유도
    tool_request_prompt = """Return ONLY JSON tool_call: 
    {"name": "get_weather", "args": {"city": "Seoul", "date": "tomorrow"}}"""
    
    print("=== Ollama 툴 호출 시뮬레이션 ===")
    tool_call_json = run_ollama("llama3.1:8b", tool_request_prompt)
    print(f"생성된 툴 호출: {tool_call_json}")
    
    # 2) 더미 툴 함수들 정의
    def fake_tools_handler(tool_call: dict) -> dict:
        """더미 툴 핸들러"""
        name = tool_call.get("name", "")
        args = tool_call.get("args", {})
        
        if name == "get_weather":
            return {"forecast": "Sunny", "temp_c": 27, "humidity": "60%"}
        elif name == "fx_usd_krw":
            return {"rate": 1385.2, "trend": "stable"}
        else:
            return {"error": "unknown tool"}
    
    # 3) JSON 파싱 후 툴 실행
    try:
        tool_data = json.loads(tool_call_json)
        result = fake_tools_handler(tool_data)
        print(f"툴 실행 결과: {result}")
        
        # 4) 결과를 바탕으로 최종 응답 생성
        final_prompt = f"""Based on this tool result: {result}
        Write a one-line weather summary in Korean."""
        
        final_response = run_ollama("llama3.1:8b", final_prompt)
        print(f"최종 응답: {final_response}")
        
    except json.JSONDecodeError:
        print("JSON 파싱 실패")

simulate_tool_calling_ollama()

=== Ollama 툴 호출 시뮬레이션 ===
생성된 툴 호출: `{"name": "get_weather", "args": {"city": "Seoul", "date": "tomorrow"}}`
JSON 파싱 실패


In [ ]:
# 1-2. OpenAI 실제 Function Calling
def openai_function_calling_demo():
    """OpenAI Function Calling 실제 구현"""
    
    # 실제 도구 함수들
    def get_weather(city: str, date: str = "today") -> dict:
        """날씨 정보 조회 (더미 데이터)"""
        weather_db = {
            "Seoul": {"forecast": "Sunny", "temp_c": 27, "humidity": "65%"},
            "Busan": {"forecast": "Cloudy", "temp_c": 24, "humidity": "70%"},
            "서울": {"forecast": "맑음", "temp_c": 27, "humidity": "65%"}
        }
        return weather_db.get(city, {"error": "City not found"})
    
    def fx_usd_krw() -> dict:
        """USD->KRW 환율 조회 (더미 데이터)"""
        return {"rate": 1385.2, "change": "+0.5%", "timestamp": "2024-01-15 09:00"}
    
    def calculate_math(expression: str) -> dict:
        """안전한 수학 계산"""
        try:
            if re.match(r'^[0-9+\-*/().\s]+$', expression):
                result = eval(expression)  # 실제로는 더 안전한 파서 사용 권장
                return {"result": result, "expression": expression}
            else:
                return {"error": "Unsupported expression"}
        except Exception as e:
            return {"error": str(e)}
    
    # Function Calling 도구 스키마 정의
    tools = [
        {
            "type": "function",
            "function": {
                "name": "get_weather",
                "description": "지정된 도시의 날씨 정보를 조회합니다",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "city": {"type": "string", "description": "도시 이름"},
                        "date": {"type": "string", "description": "날짜 (오늘/내일/yyyy-mm-dd)"}
                    },
                    "required": ["city"]
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "fx_usd_krw",
                "description": "USD에서 KRW로의 현재 환율을 조회합니다",
                "parameters": {"type": "object", "properties": {}}
            }
        },
        {
            "type": "function", 
            "function": {
                "name": "calculate_math",
                "description": "수학 계산을 수행합니다",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "expression": {"type": "string", "description": "계산할 수식"}
                    },
                    "required": ["expression"]
                }
            }
        }
    ]
    
    print("=== OpenAI Function Calling 데모 ===")
    
    # 테스트 케이스들
    test_queries = [
        "서울 날씨 확인해줘",
        "USD-KRW 환율이랑 25 * 47 계산 결과 알려줘",
        "부산 날씨와 현재 환율로 여행 예산 조언해줘"
    ]
    
    for i, query in enumerate(test_queries, 1):
        print(f"\n--- 테스트 {i}: {query} ---")
        
        # 첫 번째 요청
        messages = [{"role": "user", "content": query}]
        
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages,
            tools=tools,
            tool_choice="auto",
        )
        
        # 도구 호출 처리
        if response.choices[0].message.tool_calls:
            messages.append(response.choices[0].message)
            
            for tool_call in response.choices[0].message.tool_calls:
                function_name = tool_call.function.name
                try:
                    arguments = json.loads(tool_call.function.arguments)
                except:
                    arguments = {}
                
                print(f"도구 호출: {function_name}({arguments})")
                
                # 실제 함수 호출
                if function_name == "get_weather":
                    result = get_weather(**arguments)
                elif function_name == "fx_usd_krw":
                    result = fx_usd_krw()
                elif function_name == "calculate_math":
                    result = calculate_math(arguments.get("expression", ""))
                else:
                    result = {"error": "Unknown function"}
                
                print(f"도구 결과: {result}")
                
                messages.append({
                    "role": "tool",
                    "tool_call_id": tool_call.id,
                    "name": function_name,
                    "content": json.dumps(result, ensure_ascii=False)
                })
            
            # 최종 응답 생성
            final_response = client.chat.completions.create(
                model="gpt-4o-mini",
                messages=messages,
            )
            
            print(f"최종 응답: {final_response.choices[0].message.content}")
        else:
            print(f"도구 호출 없음: {response.choices[0].message.content}")

openai_function_calling_demo()

=== OpenAI Function Calling 데모 ===

--- 테스트 1: 서울 날씨 확인해줘 ---
도구 호출: get_weather({'city': '서울', 'date': '오늘'})
도구 결과: {'forecast': '맑음', 'temp_c': 27, 'humidity': '65%'}
최종 응답: 오늘 서울 날씨는 맑음입니다. 기온은 약 27°C, 습도는 65%입니다.  
야외 활동하기 좋은 날이니 가벼운 옷차림을 추천드리고, 자외선 차단제는 챙기세요. 우산은 필요하지 않을 것 같습니다.  
다른 시간대 예보나 내일 날씨도 알려드릴까요?

--- 테스트 2: USD-KRW 환율이랑 25 * 47 계산 결과 알려줘 ---
도구 호출: fx_usd_krw({})
도구 결과: {'rate': 1385.2, 'change': '+0.5%', 'timestamp': '2024-01-15 09:00'}
도구 호출: calculate_math({'expression': '25 * 47'})
도구 결과: {'result': 1175, 'expression': '25 * 47'}
최종 응답: USD-KRW 환율: 1,385.2원 (변동 +0.5%, 기준 시각 2024-01-15 09:00)  
25 × 47 = 1,175

더 최신 시세가 필요하시면 알려주세요.

--- 테스트 3: 부산 날씨와 현재 환율로 여행 예산 조언해줘 ---
도구 호출: get_weather({'city': '부산', 'date': '오늘'})
도구 결과: {'error': 'City not found'}
도구 호출: fx_usd_krw({})
도구 결과: {'rate': 1385.2, 'change': '+0.5%', 'timestamp': '2024-01-15 09:00'}
최종 응답: 먼저 알려드린 환율(및 날씨 조회) 상태에 대해 안내드립니다.
- 환율(제가 조회한 값): 1 USD = 1,385.2 KRW (출처 타임스탬프: 2024-01-15 09:00). 이 값은 최신이

## 2. Multiple Chains

**핵심 아이디어**: 파이프라인 연결 - 정제→요약→분류→행동추천

**패턴**:
- 팬아웃: 동일 입력→여러 모델/프롬프트 병렬
- 팬인: 여러 결과→집계/투표

**설계 포인트**:
- 각 단계의 입출력 스키마 고정(JSON)
- 에러 분기(누락 필드→재질문/기본값)
- 캐시/재사용: 반복 호출 절감

In [6]:
# 2-1. Ollama Multiple Chains (쉘 파이프라인 시뮬레이션)
def ollama_multiple_chains():
    """Ollama를 사용한 다단계 체인 처리"""
    
    input_text = "배송이 너무 늦었고, 포장도 찢어져 왔습니다. 환불 절차 알려주세요. 정말 화가 나네요."
    
    print("=== Ollama Multiple Chains 처리 ===")
    print(f"입력: {input_text}")
    
    # Stage 1: 요약 (3문장)
    stage1_prompt = f"Summarize in Korean in 3 sentences: --- {input_text} ---"
    summary = run_ollama("llama3.1:8b", stage1_prompt)
    print(f"\n[Stage 1] 요약: {summary}")
    
    # Stage 2: 의도 분류 (JSON)
    stage2_prompt = f"""Classify intent -> refund|exchange|question. Return JSON {{"intent": "...", "confidence": 0.0-1.0}}.
Text: --- {input_text} ---"""
    classification = run_ollama("llama3.1:8b", stage2_prompt)
    print(f"\n[Stage 2] 의도 분류: {classification}")
    
    # Stage 3: 감정 분석
    stage3_prompt = f"""Analyze emotion intensity -> low|medium|high. Return JSON {{"emotion": "...", "intensity": "...", "keywords": [...]}}.
Text: --- {input_text} ---"""
    emotion = run_ollama("llama3.1:8b", stage3_prompt)
    print(f"\n[Stage 3] 감정 분석: {emotion}")
    
    # Stage 4: 통합 대응문 생성
    stage4_prompt = f"""You are a CS agent. Create a response based on:
Summary: {summary}
Intent: {classification}
Emotion: {emotion}

Write a polite 3-sentence response in Korean:
1) Acknowledge the emotion
2) Apologize if needed
3) Provide clear next step"""
    
    final_response = run_ollama("llama3.1:8b", stage4_prompt)
    print(f"\n[Stage 4] 최종 대응: {final_response}")
    
    return {
        "original": input_text,
        "summary": summary,
        "classification": classification,
        "emotion": emotion,
        "response": final_response
    }

ollama_result = ollama_multiple_chains()

=== Ollama Multiple Chains 처리 ===
입력: 배송이 너무 늦었고, 포장도 찢어져 왔습니다. 환불 절차 알려주세요. 정말 화가 나네요.

[Stage 1] 요약: 배송이 매우늦게 도착했으며 제품의 포장이 찢어져있었습니다. 환불을 처리하려면 어떻게 해야할까요? 정말 기대가 안된 일이었다는 생각에 큰 불편함을 느끼고 있습니다.

[Stage 2] 의도 분류: Based on the text, I would classify the intent as:

{"intent": "refund", "confidence": 0.9}

Here's my reasoning:

* The user mentions that the delivery was delayed and the packaging was damaged ("배송이 너무 늦었고, 포장도 찢어져 왔습니다."), which suggests a problem with the product or service.
* The user explicitly asks for the refund procedure ("환불 절차 알려주세요."), indicating a clear intent to request a refund.
* The tone of the text is negative and frustrated ("정말 화가 나네요"), but this doesn't necessarily affect the classification, as it's still focused on requesting a refund.

Note that the confidence level is 0.9 because there's no explicit mention of an exchange or any other specific issue, so while it's likely that the user wants a refund, it's not a certainty. However, based on the context an

In [3]:
# 2-2. OpenAI Multiple Chains (오케스트레이터 패턴)
def openai_multiple_chains():
    """OpenAI를 사용한 고급 다단계 체인 처리"""
    
    def chat_stage(prompt: str, **kwargs):
        """단일 스테이지 실행"""
        return client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[{"role": "user", "content": prompt}],
            **kwargs
        ).choices[0].message.content
    
    # 테스트 케이스들
    test_cases = [
        "배송이 너무 늦었고, 포장도 찢어져 왔습니다. 환불 절차 알려주세요.",
        "제품이 정말 좋네요! 다음에도 주문하고 싶어요. 할인 혜택 있나요?",
        "사이즈가 안 맞아서 교환하고 싶은데, 어떻게 해야 하나요?"
    ]
    
    print("=== OpenAI Multiple Chains 처리 ===")
    
    results = []
    
    for i, text in enumerate(test_cases, 1):
        print(f"\n--- 케이스 {i}: {text} ---")
        
        # Stage 1: 언어 감지 및 정규화
        stage1 = chat_stage(
            f"Detect language and normalize text. Return JSON {{\"language\": \"...\", \"normalized\": \"...\", \"length\": number}}.\nText: --- {text} ---",
            max_completion_tokens=150
        )
        print(f"Stage 1 (정규화): {stage1}")
        
        # Stage 2: 병렬 분석 (감성 + 의도)
        stage2a = chat_stage(
            f"Sentiment analysis. Return JSON {{\"sentiment\": \"positive|negative|neutral\", \"confidence\": 0.0-1.0, \"keywords\": [...]}}.\nText: --- {text} ---",
            max_completion_tokens=120
        )
        
        stage2b = chat_stage(
            f"Intent classification. Return JSON {{\"intent\": \"refund|exchange|question|compliment\", \"urgency\": \"low|medium|high\"}}.\nText: --- {text} ---",
            max_completion_tokens=100
        )
        
        print(f"Stage 2a (감성): {stage2a}")
        print(f"Stage 2b (의도): {stage2b}")
        
        # Stage 3: 통합 분석 및 전략 수립
        stage3 = chat_stage(
            f"""Based on the analysis results, create a response strategy.
Normalization: {stage1}
Sentiment: {stage2a}
Intent: {stage2b}

Return JSON:
{{"strategy": "empathize|celebrate|inform|escalate", "tone": "apologetic|friendly|neutral|urgent", "actions": [...]}}""",
            max_completion_tokens=200
        )
        print(f"Stage 3 (전략): {stage3}")
        
        # Stage 4: 최종 응답 생성
        stage4 = chat_stage(
            f"""Generate final customer service response in Korean based on:
Strategy: {stage3}
Original message: {text}

Requirements:
- 2-3 sentences
- Professional and empathetic tone
- Include specific next steps
- Match the determined strategy and tone""",
            max_completion_tokens=200
        )
        print(f"Stage 4 (최종응답): {stage4}")
        
        results.append({
            "input": text,
            "normalization": stage1,
            "sentiment": stage2a,
            "intent": stage2b,
            "strategy": stage3,
            "response": stage4
        })
    
    return results

openai_results = openai_multiple_chains()

=== OpenAI Multiple Chains 처리 ===

--- 케이스 1: 배송이 너무 늦었고, 포장도 찢어져 왔습니다. 환불 절차 알려주세요. ---
Stage 1 (정규화): ```json
{
  "language": "Korean",
  "normalized": "배송이 너무 늦었고, 포장도 찢어져 왔습니다. 환불 절차 알려주세요.",
  "length": 39
}
```
Stage 2a (감성): ```json
{
  "sentiment": "negative",
  "confidence": 0.85,
  "keywords": ["배송", "늦었고", "포장", "찢어져", "환불", "절차"]
}
```
Stage 2b (의도): ```json
{
  "intent": "refund",
  "urgency": "high"
}
```
Stage 3 (전략): ```json
{
  "strategy": "empathize|inform|escalate",
  "tone": "apologetic|urgent",
  "actions": [
    "Acknowledge the customer's frustration due to delayed delivery and damaged packaging.",
    "Provide clear instructions on the refund process.",
    "Offer direct contact information for a customer service representative who can assist further.",
    "Prioritize this case for a quick resolution."
  ]
}
```
Stage 4 (최종응답): 안녕하세요. 배송 지연과 포장이 손상된 점 정말 죄송합니다. 환불 절차에 대해서는 저희 고객 서비스팀에 직접 연락해 주시면 신속하게 도와드리겠습니다. 아래 연락처로 연락 주시면, 우선적으로 이 건을 처리하도록 하겠습니다. 감사합니다.

---

In [4]:
# 2-3. Fan-out/Fan-in 패턴 (병렬 처리 + 결과 집계)
def fan_out_fan_in_pattern():
    """팬아웃/팬인 패턴으로 다중 관점 분석"""
    
    input_text = "이 제품은 가격 대비 성능이 좋지만, 디자인이 조금 아쉬워요. 전반적으로는 만족합니다."
    
    print("=== Fan-out/Fan-in 패턴 ===")
    print(f"입력: {input_text}")
    
    # Fan-out: 동일 입력을 여러 관점으로 분석
    perspectives = {
        "quality_focus": "Analyze from quality perspective. Focus on performance, durability, reliability.",
        "price_focus": "Analyze from price/value perspective. Focus on cost-effectiveness, worth.",
        "design_focus": "Analyze from design/UX perspective. Focus on aesthetics, usability.",
        "satisfaction_focus": "Analyze from overall satisfaction perspective. Focus on recommendation likelihood."
    }
    
    fan_out_results = {}
    
    for perspective_name, instruction in perspectives.items():
        prompt = f"""{instruction}
        
Return JSON: {{"aspect": "{perspective_name}", "rating": 1-5, "comment": "...", "keywords": [...]}}

Review: --- {input_text} ---"""
        
        result = run_openai(prompt, max_completion_tokens=150)
        fan_out_results[perspective_name] = result
        print(f"\n[{perspective_name}]: {result}")
    
    # Fan-in: 여러 관점 결과를 통합
    fan_in_prompt = f"""Aggregate these perspective analyses into a final assessment:
    
{json.dumps(fan_out_results, indent=2, ensure_ascii=False)}

Return JSON:
{{
  "overall_rating": 1-5,
  "key_strengths": [...],
  "key_weaknesses": [...],
  "recommendation": "...",
  "confidence": 0.0-1.0
}}"""
    
    final_result = run_openai(fan_in_prompt, max_completion_tokens=250)
    print(f"\n[통합 결과]: {final_result}")
    
    return {
        "input": input_text,
        "fan_out": fan_out_results,
        "fan_in": final_result
    }

fan_result = fan_out_fan_in_pattern()

=== Fan-out/Fan-in 패턴 ===
입력: 이 제품은 가격 대비 성능이 좋지만, 디자인이 조금 아쉬워요. 전반적으로는 만족합니다.

[quality_focus]: ```json
{
  "aspect": "quality_focus",
  "rating": 4,
  "comment": "The product offers good performance for its price, indicating acceptable quality, but there are concerns regarding the design, which could impact durability and reliability over time.",
  "keywords": ["performance", "durability", "reliability", "value for money", "design"]
}
```

[price_focus]: ```json
{
  "aspect": "price_focus",
  "rating": 4,
  "comment": "The product offers good performance for its price, though the design leaves something to be desired. Overall, it is satisfactory in terms of value.",
  "keywords": ["cost-effectiveness", "performance", "design", "satisfaction"]
}
```

[design_focus]: ```json
{
  "aspect": "design_focus",
  "rating": 3,
  "comment": "The product offers good performance for the price, but the design could be improved, which affects its overall appeal. While the user is generally satisfie

## 3. Meta-Prompting

**핵심 아이디어**: 모델이 프롬프트 자체를 진단/개선

**절차**:
1. 초기 프롬프트 제시
2. 개선 포인트 도출
3. 개선안 제시
4. 개선안으로 실행

**체크리스트**:
- 목표/산출형식/제약/예시/실패 시 재질의 포함 여부
- 금칙어, 길이 제한, 스키마/포맷, 평가 기준

In [15]:
# 3-1. Ollama Meta-Prompting
def ollama_meta_prompting():
    """Ollama를 사용한 메타 프롬프팅"""
    
    initial_prompt = "Classify sentiment of a review."
    
    print("=== Ollama Meta-Prompting ===")
    print(f"초기 프롬프트: {initial_prompt}")
    
    # 1) 개선 요청
    improvement_request = f"""Improve this prompt for reliability & JSON output. 
Add schema, constraints, and clarifying questions rule:

--- {initial_prompt} ---

Make it robust for production use with:
1. Clear output format specification
2. Error handling guidelines  
3. Constraints on response length
4. Confidence scoring
5. Edge case handling

Return the improved prompt only."""
    
    improved_prompt = run_ollama("llama3.1:8b", improvement_request)
    print(f"\n개선된 프롬프트:\n{improved_prompt}")
    
    # 2) 개선된 프롬프트로 실제 테스트
    test_reviews = [
        "포장이 엉망이었고 배송도 늦었습니다.",
        "제품은 괜찮은데 가격이 좀 비싸요.",
        "완벽합니다! 추천해요!"
    ]
    
    print("\n=== 개선된 프롬프트 테스트 ===")
    
    for i, review in enumerate(test_reviews, 1):
        test_prompt = f"""{improved_prompt}

Review: --- {review} ---"""
        
        result = run_ollama("llama3.1:8b", test_prompt)
        print(f"\n테스트 {i} ({review}):")
        print(f"결과: {result}")
        
        # JSON 파싱 가능성 체크
        is_valid_json = safe_json_parse(result) is not None
        print(f"JSON 유효성: {is_valid_json}")
    
    return improved_prompt

improved_ollama = ollama_meta_prompting()

=== Ollama Meta-Prompting ===
초기 프롬프트: Classify sentiment of a review.

개선된 프롬프트:
Here is the improved prompt:

--- Classify sentiment of a review as either positive, negative, or neutral, and provide a corresponding confidence score (on a scale of 0-1).

**Output Format:**

* Return a JSON object with the following fields:
	+ `sentiment`: string (positive, negative, or neutral)
	+ `confidence`: float (confidence score between 0 and 1)
	+ `error`: string (optional, describing any error that occurred during processing)

**Constraints:**

* The input review text must be at least 10 characters long and not exceed 500 characters.
* The response length must not exceed 1024 bytes.

**Error Handling Guidelines:**

* If the input review is empty or exceeds the character limit, return an `error` field with a descriptive message.
* If there is any issue with processing the review (e.g., network error), return an `error` field with a generic error message.

**Edge Case Handling:**

* Handle cases

In [5]:
# 3-2. OpenAI Meta-Prompting (더 정교한 개선)
def openai_meta_prompting():
    """OpenAI를 사용한 고급 메타 프롬프팅"""
    
    initial_prompts = [
        "Classify sentiment of a review.",
        "Summarize this text.",
        "Answer the question about the document."
    ]
    
    print("=== OpenAI Meta-Prompting ===")
    
    improved_prompts = []
    
    for i, initial in enumerate(initial_prompts, 1):
        print(f"\n--- 프롬프트 {i} 개선 ---")
        print(f"초기: {initial}")
        
        # 1) 프롬프트 분석 및 개선 계획
        analysis_prompt = f"""Analyze this prompt for weaknesses and create an improvement plan:
        
Prompt: "{initial}"

Analyze:
1. Clarity issues
2. Missing constraints
3. Output format problems
4. Edge case handling
5. Scalability concerns

Return JSON:
{{
  "weaknesses": [...],
  "improvement_plan": [...],
  "priority_fixes": [...]
}}"""
        
        analysis = run_openai(analysis_prompt, max_completion_tokens=300)
        print(f"\n분석 결과: {analysis}")
        
        # 2) 개선된 프롬프트 생성
        improvement_prompt = f"""Based on this analysis, create a robust, production-ready prompt:

Analysis: {analysis}
Original: "{initial}"

Create an improved prompt that:
1. Has clear input/output specifications
2. Includes JSON schema
3. Has error handling instructions
4. Specifies constraints (length, format, etc.)
5. Includes confidence scoring
6. Handles edge cases

Return only the improved prompt text."""
        
        improved = run_openai(improvement_prompt, max_completion_tokens=400)
        print(f"\n개선됨: {improved}")
        
        improved_prompts.append({
            "original": initial,
            "analysis": analysis,
            "improved": improved
        })
    
    # 3) 개선된 프롬프트들의 품질 평가
    print("\n=== 개선 품질 평가 ===")
    
    for i, prompt_data in enumerate(improved_prompts, 1):
        evaluation_prompt = f"""Rate the improvement quality of this prompt transformation:

Original: "{prompt_data['original']}"
Improved: "{prompt_data['improved']}"

Rate (1-10) on:
- Clarity improvement
- Robustness
- Production readiness
- Completeness

Return JSON:
{{
  "clarity_score": 1-10,
  "robustness_score": 1-10,
  "production_score": 1-10,
  "completeness_score": 1-10,
  "overall_score": 1-10,
  "comments": "..."
}}"""
        
        evaluation = run_openai(evaluation_prompt, max_completion_tokens=250)
        print(f"\n프롬프트 {i} 평가: {evaluation}")
    
    return improved_prompts

improved_openai = openai_meta_prompting()

=== OpenAI Meta-Prompting ===

--- 프롬프트 1 개선 ---
초기: Classify sentiment of a review.

분석 결과: Here's an analysis of the prompt "Classify sentiment of a review" along with an improvement plan:

```json
{
  "weaknesses": [
    "Clarity issues: The prompt does not specify what types of sentiment categories should be used (e.g., positive, negative, neutral).",
    "Missing constraints: There are no limitations provided regarding the length or format of the review text.",
    "Output format problems: The expected format of the output (e.g., a label or a score) is not specified.",
    "Edge case handling: There is no guidance on how to treat ambiguous reviews or those with mixed sentiments.",
    "Scalability concerns: The prompt does not address how the system should handle large volumes of reviews or varying lengths of text."
  ],
  "improvement_plan": [
    "Clarify sentiment categories: Specify a clear set of sentiment categories (e.g., positive, negative, neutral, mixed).",
    "Add cons

In [19]:
# 3-3. 프롬프트 버전 관리 시스템
class PromptVersionManager:
    """프롬프트 버전 관리 및 A/B 테스트"""
    
    def __init__(self):
        self.versions = {}
        self.test_results = {}
    
    def add_version(self, name: str, version: str, prompt: str, metadata: dict = None):
        """새 프롬프트 버전 추가"""
        key = f"{name}_v{version}"
        self.versions[key] = {
            "prompt": prompt,
            "metadata": metadata or {},
            "created_at": time.time()
        }
        print(f"Version {key} added")
    
    def run_ab_test(self, name: str, versions: list, test_cases: list):
        """A/B 테스트 실행"""
        results = {}
        
        for version in versions:
            key = f"{name}_v{version}"
            if key not in self.versions:
                continue
                
            version_results = []
            prompt_template = self.versions[key]["prompt"]
            
            for test_case in test_cases:
                # 프롬프트에 테스트 케이스 삽입
                full_prompt = prompt_template.replace("{input}", test_case["input"])
                
                start_time = time.time()
                result = run_openai(full_prompt, temperature=0.1, max_tokens=200)
                end_time = time.time()
                
                # 결과 평가
                score = self._evaluate_result(result, test_case.get("expected"))
                
                version_results.append({
                    "input": test_case["input"],
                    "output": result,
                    "score": score,
                    "latency": end_time - start_time,
                    "json_valid": safe_json_parse(result) is not None
                })
            
            results[key] = version_results
        
        # 결과 분석
        analysis = self._analyze_ab_results(results)
        self.test_results[f"{name}_ab_{int(time.time())}"] = {
            "results": results,
            "analysis": analysis
        }
        
        return analysis
    
    def _evaluate_result(self, result: str, expected: dict = None) -> float:
        """결과 품질 평가 (간단한 버전)"""
        score = 0.0
        
        # JSON 유효성 (+30점)
        if safe_json_parse(result):
            score += 0.3
        
        # 길이 적절성 (+20점)
        if 50 <= len(result) <= 500:
            score += 0.2
        
        # 한글 포함 여부 (+20점)
        if re.search(r'[가-힣]', result):
            score += 0.2
        
        # 기본 구조 (+30점)
        if any(keyword in result.lower() for keyword in ['label', 'confidence', 'reason']):
            score += 0.3
        
        return min(1.0, score)
    
    def _analyze_ab_results(self, results: dict) -> dict:
        """A/B 테스트 결과 분석"""
        analysis = {}
        
        for version, version_results in results.items():
            scores = [r["score"] for r in version_results]
            latencies = [r["latency"] for r in version_results]
            json_valid_rate = sum(1 for r in version_results if r["json_valid"]) / len(version_results)
            
            analysis[version] = {
                "avg_score": sum(scores) / len(scores),
                "avg_latency": sum(latencies) / len(latencies),
                "json_valid_rate": json_valid_rate,
                "total_tests": len(version_results)
            }
        
        # 최고 성능 버전 선정
        best_version = max(analysis.keys(), 
                         key=lambda x: analysis[x]["avg_score"] * analysis[x]["json_valid_rate"])
        analysis["best_version"] = best_version
        
        return analysis

# 버전 관리 시스템 테스트
def test_version_manager():
    manager = PromptVersionManager()
    
    # 프롬프트 버전들 등록
    manager.add_version(
        "sentiment", "1.0", 
        "Classify sentiment: {input}",
        {"author": "initial", "notes": "Basic version"}
    )
    
    manager.add_version(
        "sentiment", "2.0",
        """Classify sentiment with confidence scoring.
Return JSON: {"label": "positive|negative|neutral", "confidence": 0.0-1.0, "reason": "..."}
Text: {input}""",
        {"author": "improved", "notes": "Added JSON structure"}
    )
    
    # 테스트 케이스
    test_cases = [
        {"input": "배송이 빠르고 좋아요!", "expected": {"label": "positive"}},
        {"input": "포장이 엉망이고 실망스러워요", "expected": {"label": "negative"}},
        {"input": "그냥 보통이에요", "expected": {"label": "neutral"}}
    ]
    
    # A/B 테스트 실행
    print("=== 프롬프트 A/B 테스트 ===")
    analysis = manager.run_ab_test("sentiment", ["1.0", "2.0"], test_cases)
    
    print("\nA/B 테스트 결과:")
    for version, metrics in analysis.items():
        if version != "best_version":
            print(f"{version}:")
            print(f"  평균 점수: {metrics['avg_score']:.3f}")
            print(f"  평균 지연: {metrics['avg_latency']:.3f}s")
            print(f"  JSON 유효율: {metrics['json_valid_rate']:.3f}")
    
    print(f"\n최고 성능: {analysis['best_version']}")
    
    return manager

version_manager = test_version_manager()

Version sentiment_v1.0 added
Version sentiment_v2.0 added
=== 프롬프트 A/B 테스트 ===

A/B 테스트 결과:
sentiment_v1.0:
  평균 점수: 0.400
  평균 지연: 2.307s
  JSON 유효율: 0.000
sentiment_v2.0:
  평균 점수: 0.867
  평균 지연: 1.561s
  JSON 유효율: 1.000

최고 성능: sentiment_v2.0


## 4. APE (Automatic Prompt Engineering)

**핵심 아이디어**: 후보 프롬프트 자동 생성→평가→선정(루프)

**구성**:
1. 후보 K개 생성
2. Dev set 평가
3. 베스트 선택
4. 버전 태깅

**평가 방식**:
- 정답형: 정확도/EM/F1
- 생성형: LM-as-Judge(0~5), 포맷 준수율

In [20]:
# 4-1. Ollama APE (간단한 버전)
def ollama_ape_demo():
    """Ollama를 사용한 간단한 APE 데모"""
    
    # Dev set 정의
    dev_set = [
        {"text": "배송이 빠르고 만족합니다", "label": "positive"},
        {"text": "불량품이 와서 화가 납니다", "label": "negative"},
        {"text": "보통 수준입니다", "label": "neutral"},
        {"text": "가격이 적당하고 품질도 괜찮아요", "label": "positive"}
    ]
    
    print("=== Ollama APE 데모 ===")
    print(f"Dev set 크기: {len(dev_set)}")
    
    # 1) 후보 프롬프트 생성 (3개)
    candidates = []
    
    for i in range(1, 4):
        generation_prompt = f"""Generate a one-line prompt to classify sentiment (positive|negative|neutral) with JSON output.
Requirements:
- Must return JSON with keys: label, reason
- Should be clear and specific
- Version {i} - make it different from previous versions

Return only the prompt text."""
        
        candidate = run_ollama("llama3.1:8b", generation_prompt)
        candidates.append(candidate)
        print(f"\n후보 {i}: {candidate}")
    
    # 2) 각 후보를 dev set으로 평가
    best_score = 0
    best_candidate = None
    
    for i, candidate in enumerate(candidates, 1):
        correct = 0
        total = len(dev_set)
        
        print(f"\n--- 후보 {i} 평가 ---")
        
        for example in dev_set:
            test_prompt = f"""{candidate}
            
Review: --- {example['text']} ---"""
            
            result = run_ollama("llama3.1:8b", test_prompt)
            
            # 라벨 추출 (간단한 파싱)
            predicted_label = "unknown"
            for label in ["positive", "negative", "neutral"]:
                if label.lower() in result.lower():
                    predicted_label = label
                    break
            
            if predicted_label == example["label"]:
                correct += 1
            
            print(f"  {example['text'][:30]}... -> {predicted_label} (정답: {example['label']})")
        
        accuracy = correct / total
        print(f"\n후보 {i} 정확도: {accuracy:.3f} ({correct}/{total})")
        
        if accuracy > best_score:
            best_score = accuracy
            best_candidate = candidate
    
    print(f"\n=== 최고 성능 프롬프트 (정확도: {best_score:.3f}) ===")
    print(best_candidate)
    
    return {
        "candidates": candidates,
        "best_prompt": best_candidate,
        "best_score": best_score,
        "dev_set": dev_set
    }

ollama_ape_result = ollama_ape_demo()

=== Ollama APE 데모 ===
Dev set 크기: 4

후보 1: Classify the sentiment of a given product review as positive, negative, or neutral based on its emotional tone and content, outputting a JSON response with 'label' and 'reason' keys to specify the classification decision.

후보 2: Classify the sentiment of the given product review as positive, negative, or neutral, providing a brief reason for your classification in JSON format: `{"label": "positive/negative/neutral", "reason": "brief_reason_here"}`.

후보 3: Classify the sentiment of a given review as positive, negative, or neutral, returning a JSON object with keys 'label' and 'reason'.

--- 후보 1 평가 ---
  배송이 빠르고 만족합니다... -> positive (정답: positive)
  불량품이 와서 화가 납니다... -> negative (정답: negative)
  보통 수준입니다... -> positive (정답: neutral)
  가격이 적당하고 품질도 괜찮아요... -> positive (정답: positive)

후보 1 정확도: 0.750 (3/4)

--- 후보 2 평가 ---
  배송이 빠르고 만족합니다... -> positive (정답: positive)
  불량품이 와서 화가 납니다... -> negative (정답: negative)
  보통 수준입니다... -> positive (정답: n

In [6]:
# 4-2. OpenAI APE (고급 버전 - LM-as-Judge 포함)
def openai_ape_advanced():
    """OpenAI를 사용한 고급 APE 시스템"""
    
    # 더 큰 Dev set
    dev_set = [
        {"text": "배송이 빠르고 만족합니다", "label": "positive"},
        {"text": "불량품이 와서 화가 납니다", "label": "negative"}, 
        {"text": "보통 수준입니다", "label": "neutral"},
        {"text": "가격이 적당하고 품질도 괜찮아요", "label": "positive"},
        {"text": "포장이 찢어져서 실망이에요", "label": "negative"},
        {"text": "특별하지 않아요. 그냥 평범해요", "label": "neutral"},
        {"text": "완벽합니다! 강력 추천해요!", "label": "positive"},
        {"text": "돈 아까워요. 환불하고 싶네요", "label": "negative"}
    ]
    
    print("=== OpenAI 고급 APE 시스템 ===")
    print(f"Dev set 크기: {len(dev_set)}")
    
    # 1) 후보 프롬프트 생성 (5개, 다양한 접근법)
    generation_strategies = [
        "Generate a direct, simple prompt for sentiment classification with JSON output.",
        "Generate a detailed prompt with examples and constraints for sentiment classification.",
        "Generate a prompt that uses role-playing (persona) for sentiment classification.", 
        "Generate a prompt with step-by-step reasoning for sentiment classification.",
        "Generate a prompt with confidence scoring and edge case handling for sentiment classification."
    ]
    
    candidates = []
    
    for i, strategy in enumerate(generation_strategies, 1):
        generation_prompt = f"""{strategy}
        
Requirements:
- Must return valid JSON with keys: {{"label": "positive|negative|neutral", "confidence": 0.0-1.0, "reason": "..."}}
- Should handle Korean text
- Should be robust and production-ready
- Include placeholder {{input}} for the text to classify

Return only the prompt text."""
        
        candidate = run_openai(generation_prompt, temperature=0.7, max_tokens=300)
        candidates.append({
            "id": i,
            "strategy": strategy,
            "prompt": candidate
        })
        
        print(f"\n후보 {i} ({strategy.split()[2]}):")
        print(f"  {candidate[:100]}...")
    
    # 2) 다중 메트릭 평가
    def evaluate_candidate(candidate_data: dict) -> dict:
        """후보 프롬프트 종합 평가"""
        prompt_template = candidate_data["prompt"]
        results = []
        
        for example in dev_set:
            # 프롬프트 실행
            full_prompt = prompt_template.replace("{input}", example["text"])
            
            try:
                response = run_openai(full_prompt, temperature=0.1, max_tokens=200)
                
                # JSON 파싱
                json_data = safe_json_parse(response)
                
                if json_data:
                    predicted = json_data.get("label", "unknown").lower()
                    confidence = json_data.get("confidence", 0.0)
                    reason = json_data.get("reason", "")
                else:
                    # Fallback: 텍스트에서 라벨 추출
                    predicted = "unknown"
                    confidence = 0.0
                    reason = ""
                    
                    for label in ["positive", "negative", "neutral"]:
                        if label in response.lower():
                            predicted = label
                            break
                
                is_correct = predicted == example["label"]
                
                results.append({
                    "input": example["text"],
                    "expected": example["label"],
                    "predicted": predicted,
                    "confidence": confidence,
                    "reason": reason,
                    "correct": is_correct,
                    "json_valid": json_data is not None,
                    "response": response
                })
                
            except Exception as e:
                results.append({
                    "input": example["text"],
                    "expected": example["label"],
                    "error": str(e),
                    "correct": False,
                    "json_valid": False
                })
        
        # 메트릭 계산
        total = len(results)
        correct = sum(1 for r in results if r.get("correct", False))
        json_valid = sum(1 for r in results if r.get("json_valid", False))
        avg_confidence = sum(r.get("confidence", 0) for r in results if "confidence" in r) / total
        
        return {
            "accuracy": correct / total,
            "json_valid_rate": json_valid / total,
            "avg_confidence": avg_confidence,
            "results": results
        }
    
    # 3) 모든 후보 평가
    evaluations = []
    
    print("\n=== 후보 평가 진행 중... ===")
    
    for candidate in candidates:
        print(f"후보 {candidate['id']} 평가 중...")
        evaluation = evaluate_candidate(candidate)
        
        # LM-as-Judge 평가 추가
        judge_prompt = f"""Rate the quality of this prompt for sentiment classification (1-10):
        
Prompt: "{candidate['prompt']}"

Consider:
- Clarity and specificity
- Output format specification
- Robustness for edge cases
- Production readiness

Return JSON: {{"score": 1-10, "reasoning": "..."}}"""
        
        judge_response = run_openai(judge_prompt, temperature=0.2, max_tokens=200)
        judge_data = safe_json_parse(judge_response)
        judge_score = judge_data.get("score", 5) if judge_data else 5
        
        evaluation["judge_score"] = judge_score / 10.0  # 0-1로 정규화
        evaluation["judge_reasoning"] = judge_data.get("reasoning", "") if judge_data else ""
        
        evaluations.append({
            "candidate": candidate,
            "evaluation": evaluation
        })
    
    # 4) 종합 점수 계산 및 최고 후보 선정
    for eval_data in evaluations:
        eval_metrics = eval_data["evaluation"]
        
        # 가중 종합 점수 (정확도 40% + JSON유효율 30% + Judge점수 30%)
        composite_score = (
            eval_metrics["accuracy"] * 0.4 +
            eval_metrics["json_valid_rate"] * 0.3 +
            eval_metrics["judge_score"] * 0.3
        )
        eval_metrics["composite_score"] = composite_score
    
    # 결과 정렬 (종합 점수 기준)
    evaluations.sort(key=lambda x: x["evaluation"]["composite_score"], reverse=True)
    
    # 5) 결과 출력
    print("\n=== APE 평가 결과 ===")
    
    for i, eval_data in enumerate(evaluations):
        candidate = eval_data["candidate"]
        metrics = eval_data["evaluation"]
        
        print(f"\n순위 {i+1}: 후보 {candidate['id']}")
        print(f"  종합점수: {metrics['composite_score']:.3f}")
        print(f"  정확도: {metrics['accuracy']:.3f}")
        print(f"  JSON유효율: {metrics['json_valid_rate']:.3f}")
        print(f"  Judge점수: {metrics['judge_score']:.3f}")
        print(f"  평균신뢰도: {metrics['avg_confidence']:.3f}")
    
    best_candidate = evaluations[0]
    print(f"\n=== 최고 성능 프롬프트 (종합점수: {best_candidate['evaluation']['composite_score']:.3f}) ===")
    print(best_candidate["candidate"]["prompt"])
    
    return {
        "candidates": candidates,
        "evaluations": evaluations,
        "best_candidate": best_candidate,
        "dev_set": dev_set
    }

ape_result = openai_ape_advanced()

=== OpenAI 고급 APE 시스템 ===
Dev set 크기: 8

후보 1 (direct,):
  Classify the sentiment of the following Korean text: {input}. Return the result in JSON format with ...

후보 2 (detailed):
  You are a sentiment classification model tasked with analyzing the sentiment of a given text input i...

후보 3 (prompt):
  You are a sentiment analysis expert tasked with classifying the sentiment of a given text in Korean....

후보 4 (prompt):
  ```json
{
  "label": "{label}",
  "confidence": {confidence},
  "reason": "The sentiment classificat...

후보 5 (prompt):
  ```json
{
  "input": "{input}",
  "output": {
    "label": "positive|negative|neutral",
    "confide...

=== 후보 평가 진행 중... ===
후보 1 평가 중...
후보 2 평가 중...
후보 3 평가 중...
후보 4 평가 중...
후보 5 평가 중...

=== APE 평가 결과 ===

순위 1: 후보 1
  종합점수: 0.940
  정확도: 1.000
  JSON유효율: 1.000
  Judge점수: 0.800
  평균신뢰도: 0.925

순위 2: 후보 4
  종합점수: 0.940
  정확도: 1.000
  JSON유효율: 1.000
  Judge점수: 0.800
  평균신뢰도: 0.890

순위 3: 후보 3
  종합점수: 0.920
  정확도: 0.875
  JSON유효율: 1.000
  Judge점

In [21]:
# 4-3. APE 결과 검증 및 최종 테스트
def validate_ape_winner():
    """APE 우승 프롬프트 검증"""
    
    if not ape_result:
        print("APE 결과가 없습니다.")
        return
    
    best_prompt = ape_result["best_candidate"]["candidate"]["prompt"]
    
    # 새로운 테스트 케이스 (Dev set과 다름)
    test_cases = [
        "이 제품 정말 최고예요! 친구들에게도 추천했어요.",
        "배송비가 너무 비싸고 포장도 부실해요. 다시는 안 사겠어요.",
        "그냥 평범한 제품이에요. 나쁘지도 좋지도 않네요.",
        "생각보다 괜찮은데 가격이 좀 아쉬워요.",
        "완전 대박! 이런 걸 찾고 있었어요!"
    ]
    
    print("=== APE 우승 프롬프트 최종 검증 ===")
    print("\n우승 프롬프트:")
    print(best_prompt)
    
    print("\n=== 새로운 테스트 케이스 결과 ===")
    
    total_tests = len(test_cases)
    json_valid_count = 0
    total_confidence = 0
    
    for i, test_case in enumerate(test_cases, 1):
        full_prompt = best_prompt.replace("{input}", test_case)
        
        result = run_openai(full_prompt, temperature=0.1, max_tokens=200)
        
        print(f"\n테스트 {i}: {test_case}")
        print(f"결과: {result}")
        
        # JSON 유효성 및 구조 검증
        json_data = safe_json_parse(result)
        if json_data:
            json_valid_count += 1
            confidence = json_data.get("confidence", 0)
            total_confidence += confidence
            
            print(f"  ✓ JSON 유효, 라벨: {json_data.get('label')}, 신뢰도: {confidence}")
        else:
            print(f"  ✗ JSON 파싱 실패")
    
    # 최종 통계
    json_success_rate = json_valid_count / total_tests
    avg_confidence = total_confidence / max(json_valid_count, 1)
    
    print(f"\n=== 최종 검증 통계 ===")
    print(f"JSON 성공률: {json_success_rate:.1%} ({json_valid_count}/{total_tests})")
    print(f"평균 신뢰도: {avg_confidence:.3f}")
    
    # 성능 기준 평가
    if json_success_rate >= 0.8 and avg_confidence >= 0.6:
        print("\n🎉 프롬프트가 프로덕션 기준을 충족합니다!")
    elif json_success_rate >= 0.6:
        print("\n⚠️ 프롬프트가 기본 기준은 충족하지만 개선이 필요합니다.")
    else:
        print("\n❌ 프롬프트가 기준을 충족하지 못합니다. 추가 개선이 필요합니다.")

validate_ape_winner()

NameError: name 'ape_result' is not defined

## 종합 실습: 통합 시스템

이 코드는 지금까지 다룬 모든 프롬프트 엔지니어링 기법들을 하나로 통합한 **고급 프롬프트 엔지니어링 시스템(AdvancedPromptSystem)**을 보여줍니다. 이 시스템은 Function Calling, Multiple Chains, Meta-Prompting 기능을 모두 포함하여, 복합적인 AI 태스크를 체계적으로 처리합니다.

시스템 구성 요소
AdvancedPromptSystem 클래스는 여러 모듈로 구성되어 있습니다.

PromptVersionManager: 이전에 설명된 클래스로, 프롬프트 버전을 관리하고 A/B 테스트를 수행하여 최적의 프롬프트를 찾는 기능을 담당합니다.

tool_registry: 시스템이 사용할 수 있는 모든 **도구(함수)**를 등록하고 관리하는 저장소입니다. 각 도구는 이름, 함수, 설명, 매개변수 스키마를 포함합니다.

chain_definitions: 여러 단계의 LLM 호출을 묶어 **파이프라인(chain)**으로 정의하는 저장소입니다. 각 파이프라인은 정해진 순서대로 LLM 호출과 도구 실행을 처리합니다.

execute_with_tools: 사용자의 요청을 받아, 등록된 도구 스키마를 OpenAI에 전달하고 Function Calling을 실행하는 핵심 메서드입니다.

execute_chain: 정의된 파이프라인을 실행하는 메서드입니다. 각 단계의 출력값을 다음 단계의 입력값으로 전달하며, 도구 사용 여부에 따라 execute_with_tools를 호출합니다.

auto_improve_prompt: 메타 프롬프팅을 사용하여 초기 프롬프트의 약점을 분석하고, 개선된 프롬프트를 자동으로 생성하는 메서드입니다.

통합 시스템 데모(demo_integrated_system)
이 함수는 위에서 정의된 AdvancedPromptSystem을 실제로 사용해봅니다.

도구 등록:
system.register_tool() 메서드를 사용하여 analyze_sentiment라는 '감성 분석' 도구를 시스템에 등록합니다. 이 도구는 실제로는 간단한 키워드 기반 분석을 수행하지만, 시스템 내에서는 하나의 '도구'로 기능합니다.

처리 체인 정의:
system.define_chain() 메서드를 사용하여 "customer_service"라는 이름의 파이프라인을 정의합니다. 이 파이프라인은 세 단계로 구성됩니다.

preprocessing: 입력 텍스트를 정규화합니다.

sentiment_analysis: 이 단계는 use_tools: True로 설정되어, LLM에게 '감성 분석' 도구(analyze_sentiment)를 사용하라고 지시합니다.

response_generation: 앞선 단계들의 결과를 바탕으로 고객 응대 메시지를 생성합니다.

체인 실행:
system.execute_chain("customer_service", test_input)를 호출하면, 시스템은 정의된 파이프라인을 순차적으로 실행합니다. 특히, 2단계에서 LLM은 analyze_sentiment 도구를 호출할 것이고, 시스템은 이 도구의 실행 결과(예: {"sentiment": "negative", "confidence": 0.8})를 받아 3단계로 전달합니다.

프롬프트 자동 개선:
system.auto_improve_prompt()를 호출하여 initial_prompt("답변해주세요.")와 몇 가지 예시를 제공합니다. 시스템은 이 정보를 바탕으로 더 구체적이고 효과적인 프롬프트를 자동으로 생성합니다.

이 시스템의 의미
이 코드는 단순한 개별 기법 데모를 넘어, 각 기법이 어떻게 현실적인 애플리케이션의 일부가 될 수 있는지 보여줍니다.

모듈화: 각 기능(도구, 체인, 버전 관리)이 별도의 모듈로 분리되어 있어, 시스템을 쉽게 확장하고 관리할 수 있습니다.

자동화: 메타 프롬프팅을 통해 최적의 프롬프트를 자동으로 생성하고, 파이프라인을 통해 복잡한 작업을 자동화하여 처리합니다.

오케스트레이션: execute_chain과 execute_execute_with_tools 메서드는 LLM 호출과 외부 도구 실행을 유기적으로 연결하는 오케스트레이터 역할을 수행합니다.

이는 LLM을 활용한 시스템 개발이 단순히 run_openai(prompt)를 호출하는 것을 넘어, 체계적인 설계와 자동화된 관리가 필요한 복잡한 엔지니어링 영역임을 시사합니다.

In [22]:
# 통합 시스템: 모든 기법을 결합한 고급 프롬프트 엔지니어링 시스템
class AdvancedPromptSystem:
    """고급 프롬프트 엔지니어링 통합 시스템"""
    
    def __init__(self):
        self.version_manager = PromptVersionManager()
        self.tool_registry = {}
        self.chain_definitions = {}
        self.ape_history = []
    
    def register_tool(self, name: str, func, description: str, parameters: dict):
        """도구 등록"""
        self.tool_registry[name] = {
            "function": func,
            "description": description,
            "parameters": parameters
        }
    
    def define_chain(self, name: str, stages: list):
        """처리 체인 정의"""
        self.chain_definitions[name] = stages
    
    def execute_with_tools(self, prompt: str, available_tools: list = None):
        """도구를 활용한 프롬프트 실행"""
        if available_tools is None:
            available_tools = list(self.tool_registry.keys())
        
        # OpenAI Function Calling 형식으로 도구 변환
        tools_schema = []
        for tool_name in available_tools:
            if tool_name in self.tool_registry:
                tool_info = self.tool_registry[tool_name]
                tools_schema.append({
                    "type": "function",
                    "function": {
                        "name": tool_name,
                        "description": tool_info["description"],
                        "parameters": tool_info["parameters"]
                    }
                })
        
        # 실행
        messages = [{"role": "user", "content": prompt}]
        
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages,
            tools=tools_schema,
            tool_choice="auto",
            temperature=0.3
        )
        
        # 도구 호출 처리
        if response.choices[0].message.tool_calls:
            messages.append(response.choices[0].message)
            
            for tool_call in response.choices[0].message.tool_calls:
                tool_name = tool_call.function.name
                if tool_name in self.tool_registry:
                    try:
                        args = json.loads(tool_call.function.arguments)
                        result = self.tool_registry[tool_name]["function"](**args)
                        
                        messages.append({
                            "role": "tool",
                            "tool_call_id": tool_call.id,
                            "name": tool_name,
                            "content": json.dumps(result, ensure_ascii=False)
                        })
                    except Exception as e:
                        messages.append({
                            "role": "tool",
                            "tool_call_id": tool_call.id,
                            "name": tool_name,
                            "content": f"Error: {str(e)}"
                        })
            
            # 최종 응답
            final_response = client.chat.completions.create(
                model="gpt-4o-mini",
                messages=messages,
                temperature=0.3
            )
            return final_response.choices[0].message.content
        else:
            return response.choices[0].message.content
    
    def execute_chain(self, chain_name: str, input_data: str):
        """정의된 체인 실행"""
        if chain_name not in self.chain_definitions:
            return f"Chain '{chain_name}' not found"
        
        stages = self.chain_definitions[chain_name]
        current_data = input_data
        stage_results = []
        
        for i, stage in enumerate(stages):
            stage_prompt = stage["prompt"].format(input=current_data)
            
            if stage.get("use_tools", False):
                result = self.execute_with_tools(
                    stage_prompt, 
                    stage.get("tools", [])
                )
            else:
                result = run_openai(
                    stage_prompt, 
                    **stage.get("params", {})
                )
            
            stage_results.append({
                "stage": i + 1,
                "name": stage.get("name", f"Stage {i+1}"),
                "input": current_data,
                "output": result
            })
            
            current_data = result
        
        return {
            "final_result": current_data,
            "stage_results": stage_results
        }
    
    def auto_improve_prompt(self, initial_prompt: str, task_examples: list):
        """메타 프롬프팅을 통한 자동 개선"""
        # 분석
        analysis_prompt = f"""Analyze this prompt and suggest improvements:
        
Prompt: "{initial_prompt}"
Task examples: {json.dumps(task_examples, ensure_ascii=False)}

Return JSON with:
{{
  "weaknesses": [...],
  "improvement_suggestions": [...],
  "improved_prompt": "..."
}}"""
        
        analysis = run_openai(analysis_prompt, temperature=0.2, max_tokens=500)
        analysis_data = safe_json_parse(analysis)
        
        if analysis_data:
            return analysis_data
        else:
            return {"error": "Failed to parse analysis", "original": initial_prompt}

# 통합 시스템 데모
def demo_integrated_system():
    """통합 시스템 데모"""
    
    system = AdvancedPromptSystem()
    
    print("=== 고급 프롬프트 엔지니어링 통합 시스템 데모 ===")
    
    # 1) 도구 등록
    def analyze_sentiment(text: str) -> dict:
        """감성 분석 도구"""
        # 간단한 키워드 기반 분석
        positive_words = ['좋', '최고', '만족', '추천', '완벽']
        negative_words = ['나쁘', '최악', '실망', '화', '문제']
        
        pos_count = sum(1 for word in positive_words if word in text)
        neg_count = sum(1 for word in negative_words if word in text)
        
        if pos_count > neg_count:
            return {"sentiment": "positive", "confidence": 0.8}
        elif neg_count > pos_count:
            return {"sentiment": "negative", "confidence": 0.8}
        else:
            return {"sentiment": "neutral", "confidence": 0.6}
    
    system.register_tool(
        "analyze_sentiment",
        analyze_sentiment,
        "텍스트의 감성을 분석합니다",
        {
            "type": "object",
            "properties": {
                "text": {"type": "string", "description": "분석할 텍스트"}
            },
            "required": ["text"]
        }
    )
    
    # 2) 처리 체인 정의
    system.define_chain("customer_service", [
        {
            "name": "preprocessing",
            "prompt": "Clean and normalize this text: {input}",
            "params": {"temperature": 0.1, "max_tokens": 100}
        },
        {
            "name": "sentiment_analysis", 
            "prompt": "Analyze the sentiment of this customer message: {input}",
            "use_tools": True,
            "tools": ["analyze_sentiment"]
        },
        {
            "name": "response_generation",
            "prompt": "Generate appropriate customer service response based on: {input}",
            "params": {"temperature": 0.3, "max_tokens": 200}
        }
    ])
    
    # 3) 테스트 실행
    test_input = "배송이 너무 늦어서 화가 나는데, 언제 받을 수 있나요?"
    
    print(f"\n테스트 입력: {test_input}")
    
    # 체인 실행
    chain_result = system.execute_chain("customer_service", test_input)
    
    print("\n=== 체인 실행 결과 ===")
    for stage in chain_result["stage_results"]:
        print(f"\n{stage['name']} (단계 {stage['stage']}):")
        print(f"  입력: {stage['input'][:50]}...")
        print(f"  출력: {stage['output'][:100]}...")
    
    print(f"\n최종 결과: {chain_result['final_result']}")
    
    # 4) 프롬프트 자동 개선
    print("\n=== 프롬프트 자동 개선 ===")
    initial_prompt = "답변해주세요."
    examples = [
        {"input": "날씨가 어때요?", "expected": "구체적인 정보 제공"},
        {"input": "추천해주세요", "expected": "명확한 추천과 이유"}
    ]
    
    improvement = system.auto_improve_prompt(initial_prompt, examples)
    
    print(f"초기 프롬프트: {initial_prompt}")
    if "improved_prompt" in improvement:
        print(f"개선된 프롬프트: {improvement['improved_prompt']}")
        print(f"개선 사항: {improvement.get('improvement_suggestions', [])}")
    
    return system

integrated_system = demo_integrated_system()

=== 고급 프롬프트 엔지니어링 통합 시스템 데모 ===

테스트 입력: 배송이 너무 늦어서 화가 나는데, 언제 받을 수 있나요?

=== 체인 실행 결과 ===

preprocessing (단계 1):
  입력: 배송이 너무 늦어서 화가 나는데, 언제 받을 수 있나요?...
  출력: 배송이 너무 늦어서 화가 나는데, 언제 받을 수 있나요? 

**Normalized Text:**
배송이 늦어서 화가 나는데, 언제 받을 수 있나요?...

sentiment_analysis (단계 2):
  입력: 배송이 너무 늦어서 화가 나는데, 언제 받을 수 있나요? 

**Normalized Tex...
  출력: The sentiment of the customer message is negative, with a confidence level of 80%. The customer expr...

response_generation (단계 3):
  입력: The sentiment of the customer message is negative,...
  출력: Subject: Update on Your Order Delivery

Dear [Customer's Name],

Thank you for reaching out to us. I...

최종 결과: Subject: Update on Your Order Delivery

Dear [Customer's Name],

Thank you for reaching out to us. I sincerely apologize for the frustration and inconvenience caused by the delay in your order delivery. I understand how important it is for you to receive your items on time.

I am currently looking into the status of your order and will provide y

## 정리 및 베스트 프랙티스

### 고급 통합 기법 요약

| 기법 | 핵심 아이디어 | 주요 장점 | 적용 상황 |
|------|---------------|-----------|----------|
| **Function Calling** | 외부 도구/API 연동 | 실시간 데이터, 정확한 계산 | 최신 정보 필요, 복잡한 연산 |
| **Multiple Chains** | 단계적 파이프라인 처리 | 복잡한 워크플로우 처리 | 다단계 분석, 의사결정 |
| **Meta-Prompting** | 프롬프트 자체 개선 | 지속적 품질 향상 | 프롬프트 최적화, 유지보수 |
| **APE** | 자동 프롬프트 생성/평가 | 객관적 성능 비교 | 대규모 최적화, A/B 테스트 |

### 실무 적용 가이드

#### 1. 시스템 설계 원칙
- **모듈화**: 각 기법을 독립적 모듈로 구현
- **확장성**: 새로운 도구/체인 쉽게 추가 가능
- **관찰성**: 각 단계별 로깅 및 메트릭 수집
- **안전성**: 도구 호출 시 보안 검증 및 예외 처리

#### 2. 성능 최적화
- **캐싱**: 반복 호출 결과 캐시
- **병렬 처리**: Fan-out 패턴으로 독립적 작업 병렬화
- **배치 처리**: 유사한 요청들 배치로 처리
- **지연 로딩**: 필요한 시점에만 리소스 로드

#### 3. 품질 보증
- **자동 테스트**: Dev/Test 셋으로 지속적 검증
- **A/B 테스트**: 새로운 프롬프트 버전 비교
- **사용자 피드백**: 실제 사용 결과 수집 및 반영
- **버전 관리**: 프롬프트 변경사항 추적

#### 4. 운영 고려사항
- **비용 모니터링**: 토큰 사용량 및 API 호출 비용 추적
- **지연 시간**: 응답 시간 SLA 설정 및 모니터링
- **오류 처리**: 실패 시 fallback 전략
- **보안**: 민감 정보 필터링 및 접근 제어